In [113]:
#!pip install -U tabula-py
#!pip install pdfminer.six
#!pip install pdfreader
#!pip install PyPDF2
!pip install pyPdf

In [2]:
#importando as bibliotecas
import bs4
import urllib.request as urllib_request
import pandas as pd
import requests

#confirmando as versões das bibliotecas
print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)
print("requests -> ", requests.__version__)

BeautifulSoup -> 4.6.3
urllib -> 3.6
pandas -> 1.1.3
requests ->  2.23.0


# **WEBSCRAPING - DIÁRIO OFICIAL DA UNIÃO**

In [88]:
from urllib.request import urlopen #acessa o site e obtem resposta
from bs4 import BeautifulSoup

DATA = '29/10/2020'
PAGINA = '1'
TOTAL = '52'
DATA_2 = '2020/10/29'
ARQUIVO = '2020_10_29_ASSINADO_do2.pdf'
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'

url = f'https://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?jornal=529&pagina={PAGINA}&data={DATA}&totalArquivos={TOTAL}'
url2 = f'https://www.jusbrasil.com.br/diarios/DOU/{DATA_2}/Secao-2'
response = urlopen(url)
html = response.read() #lê o html

In [84]:
requests.get(url2, {'User-Agent' : USER_AGENT})

<Response [403]>

In [4]:
#usando o BeautifulSoup para deixar o arquivo mais fácil de ler
soup = BeautifulSoup(html, 'html.parser')
soup


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Sumário disponível no SGPUB" name="description"/>
<meta content="Página 1 do Diário Oficial da União - Seção 2, número 208, de 29/10/2020 - Imprensa Nacional" name="keywords"/>
<title>Página 1 do Diário Oficial da União - Seção 2, número 208, de 29/10/2020 - Imprensa Nacional</title>
<link href="../../img/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<script src="../../js/INPDFViewer.js" type="text/javascript"></script>
</head>
<frameset border="1" frameborder="no" framespacing="1" rows="58,*">
<frame name="controlador" scrolling="no" src="../visualiza/navegaJornalSumario.jsp?jornal=529&amp;pagina=1&amp;data=29/10/2020&amp;totalArquivos=52"/>
<frame name="visualizador" src="../../servlet/INPDFViewer?jornal=529&amp;pagina=1&amp;data=29/10/2020&amp;captchafield=firstAccess"/>
</frameset>
<noframes></noframes>
</html>

In [7]:
soup.find('title').get_text()

'Página 1 do Diário Oficial da União - Seção 2, número 208, de 29/10/2020 - Imprensa Nacional'

In [89]:
#tentando ler um pdf
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
 
text = extract_text(ARQUIVO, laparams=LAParams())
 
print(text)


REPÚBLICA FEDERATIVA DO BRASIL • IMPRENSA NACIONALAno LXI Nº 208Brasília - DF, quinta-feira, 29 de outubro de 2020ISSN 1677-70502Documento assinado digitalmente conforme MP nº 2.200-2 de 24/08/2001,que institui a Infraestrutura de Chaves Públicas Brasileira - ICP-Brasil.Este documento pode ser verificado no endereço eletrônicohttp://www.in.gov.br/autenticidade.html, pelo código 052920201029000011Atos do Poder Executivo.......................................................................................................... 1Presidência da República.......................................................................................................... 1Ministério da Agricultura, Pecuária e Abastecimento............................................................ 2Ministério da Cidadania............................................................................................................ 5Ministério da Defesa........................................................................

In [60]:
#vamos usar o nltk: https://www.nltk.org/
import nltk
nltk.download('punkt') #incluindo a biblioteca de acordo com o erro dado na questão abaixo

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [61]:
#queremos tokens de palavras e tokens de pontuação
#redefinindo o palavras_separadas e testando com o texto exemplo
palavras_separadas = nltk.tokenize.word_tokenize(text, language='portuguese')

In [62]:
len(palavras_separadas)

97267

In [63]:
palavras_separadas[:5]

['REPÚBLICA', 'FEDERATIVA', 'DO', 'BRASIL', '•']

In [64]:
#vamos fazer uma função para separar as palavras das pontuações
def separa_palavras(lista_tokens):
  #criando uma lista de palavras
  lista_palavras=[]
  #vamos percorrer a lista e perguntar se o token é uma palavra
  for token in lista_tokens:
    if token.isalpha():
      lista_palavras.append(token)
  return lista_palavras

In [65]:
lista = separa_palavras(palavras_separadas)

In [66]:
lista[:5]

['REPÚBLICA', 'FEDERATIVA', 'DO', 'BRASIL', 'IMPRENSA']

In [67]:
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams
output_string = StringIO()
with open(ARQUIVO, 'rb') as fin:
  extract_text_to_fp(fin, output_string, laparams=LAParams(),
  output_type='html', codec=None)

In [76]:
text


'REPÚBLICA FEDERATIVA DO BRASIL • IMPRENSA NACIONALAno LXI Nº 208Brasília - DF, quinta-feira, 29 de outubro de 2020ISSN 1677-70502Documento assinado digitalmente conforme MP nº 2.200-2 de 24/08/2001,que institui a Infraestrutura de Chaves Públicas Brasileira - ICP-Brasil.Este documento pode ser verificado no endereço eletrônicohttp://www.in.gov.br/autenticidade.html, pelo código 052920201029000011Atos do Poder Executivo.......................................................................................................... 1Presidência da República.......................................................................................................... 1Ministério da Agricultura, Pecuária e Abastecimento............................................................ 2Ministério da Cidadania............................................................................................................ 5Ministério da Defesa.......................................................................

In [55]:
#usando outra lib
import pdfreader

In [68]:
from pdfreader import PDFDocument, SimplePDFViewer

In [69]:
#lendo o documento com o pdfreader
fd = open(ARQUIVO, "rb")
doc = PDFDocument(fd)

In [70]:
#vendo quantas páginas tem o documento
all_pages = [p for p in doc.pages()]
len(all_pages)


52

In [72]:
#criando um visualizador de pdf
viewer = SimplePDFViewer(fd)

In [77]:
#visualizando uma página específica
viewer.navigate(1)
viewer.canvas.text_content

''

In [91]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open(ARQUIVO, 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

texto = output_string.getvalue()

In [92]:
separado = texto.split()

In [95]:
separado[100:110]


['Humanos.................................................',
 '34Ministério',
 'da',
 'Saúde................................................................................................................',
 '34Ministério',
 'do',
 'Turismo.............................................................................................................',
 '42Controladoria-Geral',
 'da',
 'União.................................................................................................']

In [103]:
import PyPDF2
from PyPDF2 import PdfFileReader
 
# Creating a pdf file object.
pdf = open(ARQUIVO, "rb")
 


In [104]:
# read pdf
read_pdf = PyPDF2.PdfFileReader(pdf)

In [105]:
#check pdf is encrypted or not
read_pdf.getIsEncrypted()

False

In [106]:
# no of pages
read_pdf.numPages

52

In [116]:
# extract text from page number 1
page1 = read_pdf.getPage(2)
page1.extractText()

''

In [117]:
!pip install pdfquery

  Created wheel for pdfquery: filename=pdfquery-0.4.3-cp36-none-any.whl size=16781 sha256=2619bb9054a39adf8923aecf1c8e028aa5a65261a78c16723fb2bffa340bdaf8
  Stored in directory: /root/.cache/pip/wheels/eb/3b/ac/b14bb31ebd3b27b1c057ea4c38a2f24981280074fd7d64438d
Successfully built pdfquery


In [118]:
import pdfquery

In [119]:
pdf = pdfquery.PDFQuery(ARQUIVO)
pdf.load()
label = pdf.pq('LTTextLineHorizontal:contains("NOMEAR")')
left_corner = float(label.attr('x0'))
bottom_corner = float(label.attr('y0'))
name = pdf.pq('LTTextLineHorizontal:in_bbox("%s, %s, %s, %s")' % (left_corner, bottom_corner-30, left_corner+150, bottom_corner)).text()
name


'(019621823-4) M E N EZ ES Conceder aposentadoria voluntária (5ª Câmara e 6ª Câmara)'

In [ ]:
pdf.doc